## * Recommendation project for people who want to open a restaurant in Berlin,Germany.

#### 1) Introduction/Business Problem 

* **Introduction**
= Berlin is the one of the very trendy cities. Berlin has been rapidly changing a lot because many people are coming for leaving and working. Hence various resaurants are openining and closing. my purpose of the data analysis is to help people's business. 


* **Problem**
= Many people can possibly think that once they open a restaurant with very fancy and tasty food in Berlin, wherever it opens, it would be a success. but the problem is that in Berlin, there are already too many competitors on even one kind of food. Personally, I already saw some people struggling to find a proper solution or information to start with their business. but people tend to rely on the some information in internet, which is not reliable. 


* **Objective**
= Hence I would like to help people's business to avoid the worst scenario. Because once people starts with a own business, probably they might need to spend the cost to setup the restaurant.Therefore , I'm going to design the recommendation data analysis by using the Foursqure data and clustering technique,ML,  And Based on the data, I'm going to recommend the best spot for their business. Kreuzberg is currently quite trendy and popular town in Berlin for people to enjoy various cultures like foods. Therefore I'm going to focus on this town to start with this project. 

#### 2) Data

There are two big categories for the data, which are location data and venue data from Foursquare. these data will be utilized for the data analaysis when to recommend the places for users who who want to find out the nice places for their businesses.  

1. Berlin post codes, geo-location information (latitude, longitude)
: basically, the data includes geo location information of all berlin area as well as post code information.
https://data.technologiestiftung-berlin.de/dataset/plz/en

2. Berlin map based on postcode information (to narrow down the range )
: actually the data is not utilized on the real code, however it is referred to narrow down the data range(area)
https://www.interkart.de/en/wall-maps/europe/germany/city-maps/berlin-postcode-area-map.html  

3. Berlin Kreuzberg map information to check
http://zip-code.en.mapawi.com/germany/10/kreisfreie-stadt-berlin/2/141/berlin-kreuzberg/10961/1330/

4. Foursquare venue information ( via API )
https://developer.foursquare.com/docs/api/venues/details

* test ( checking out the data )

In [10]:
import json
import pandas as pd
lst_dict = []


with open('./data/postcodes_berlin.geojson') as f:
    data = json.load(f)

df = pd.DataFrame(columns=['postcode','latitude','longitude'])

for feature in data['features']:
    # only check Kreuzberg area in Berlin 
    if feature['properties']['spatial_name'] in ('10961', '10963', '10965', '10967', '10969', '10997', '10999'):  
        for position in feature['geometry']['coordinates']:
            for i in position:
                df = df.append({'postcode': feature['properties']['spatial_name'],'latitude':i[1],'longitude':i[0]}, ignore_index=True)

df.shape

(1361, 3)

In [16]:
df

,postcode,latitude,longitude
0,10961,52.495077,13.385328
1,10961,52.494434,13.385268
2,10961,52.494110,13.387134
3,10961,52.493657,13.386930
4,10961,52.493645,13.386999
...,...,...,...
1356,10999,52.495644,13.412522
1357,10999,52.495618,13.412517
1358,10999,52.495578,13.412509
1359,10999,52.495479,13.412456


* Representating the Kreuzberg area by using the **postcodes** and **coordinations**

In [15]:
#BERLIN Kreuzberg position
latitude=52.4992
longitude=13.4307

import folium

map_berlin_kreuzberg = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat,lng,postcode in zip(df['latitude'], df['longitude'], df['postcode']):
    label='{}'.format(postcode)
    label=folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_collor='#3186cc',
        fill_opacity=0.2,
        parse_html=False).add_to(map_berlin_kreuzberg)

map_berlin_kreuzberg